In [1]:
from mido import MidiFile

In [2]:
## Array con archivos cargados de test

files_mid = ['Concerto Grosso op n8 1-2mov','Sonata n5 op25 1mov','Sonata n5 op25 2mov', \
             'bach_inventions_772_[free]_(c)simonetto','Contradance n1','Contradance n3', \
             'K02 Minuet in F','K03 Allegro in Bb','K04 Minuet in F','Piano Concerto n1 2mov',\
             'racelegy','rach1-1','rach1-3','rach2-1']

file_num = 10
files_mid[file_num]

'racelegy'

In [3]:
## Levanta el archivo midi, devuelve cantidad de tracks y mensajes contenidos

mid = MidiFile(files_mid[file_num] + '.mid', clip=True)
print(mid)
print('ticks por beat:', mid.ticks_per_beat)
duracion_tema = mid.length
print('segundos:', duracion_tema)


<midi file 'racelegy.mid' type 1, 2 tracks, 5603 messages>
ticks por beat: 120
segundos: 247.10111437499953


In [4]:
## Variables globales para el ploteo

plot_from_second = 0
plot_to_second = 60 #duracion_tema

type 0 (single track): all messages are saved in one track

type 1 (synchronous): all tracks start at the same time

type 2 (asynchronous): each track is independent of the others

In [5]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
midi_notes = pd.read_csv('midi_notes.csv',sep=";")
midi_notes

,note_number,Octave,note_name,note
0,0,0,C,C0
1,1,0,C#,C#0
2,2,0,D,D0
3,3,0,D#,D#0
4,4,0,E,E0
...,...,...,...,...
123,123,10,D#,D#10
124,124,10,E,E10
125,125,10,F,F10
126,126,10,F#,F#10


In [6]:
## Itera sobre los mensaje midi de los tracks con mensjaes y genera un dataframe con notas, velocity, tick_start tick_stop

tema_df = pd.DataFrame(columns=['note_num', 'start', 'end', 'velocity','time_dif', 'track_name']) 

count_notes = 0
last_note_on = 0
info = []
controls = []
n_track = 0
for track in mid.tracks:
    n_track = n_track + 1
    track_name = track.name + str(n_track)
    info.append(track_name)
    if len(track) > 50:
        time = 0
        has_note_off = any(msg.type == 'note_off' for msg in track)
        print('mensajes:',len(track))
        for msg in track:          
            if msg.type in ['note_on', 'note_off']:
                time = time + msg.time
                if (has_note_off and (msg.type == 'note_on')) or (not has_note_off and msg.velocity > 0):
                    tema_df.loc[count_notes] = [msg.note, time,-1, msg.velocity, time - last_note_on, track_name]
                    count_notes = count_notes + 1
                    last_note_on = time
                else:
                    tema_df.loc[(tema_df.note_num == msg.note) & (tema_df.end == -1),'end'] = time
            else:
                if (msg.type == 'control_change'):
                    controls.append(msg.control)
                #elif (msg.type == 'set_tempo'):
                 #   print(msg)

print('info tracks:', info)
print('tipo note off:', has_note_off)
print('cantidad eventos pedal:', len(controls))
display(tema_df)  

mensajes: 1784
<meta message set_tempo tempo=571429 time=0>
<meta message set_tempo tempo=568720 time=3045>
<meta message set_tempo tempo=566038 time=2>
<meta message set_tempo tempo=563380 time=3>
<meta message set_tempo tempo=560748 time=2>
<meta message set_tempo tempo=555556 time=3>
<meta message set_tempo tempo=552995 time=2>
<meta message set_tempo tempo=550459 time=3>
<meta message set_tempo tempo=545455 time=2>
<meta message set_tempo tempo=542986 time=60>
<meta message set_tempo tempo=545455 time=8>
<meta message set_tempo tempo=547945 time=7>
<meta message set_tempo tempo=550459 time=8>
<meta message set_tempo tempo=552995 time=7>
<meta message set_tempo tempo=555556 time=8>
<meta message set_tempo tempo=558140 time=7>
<meta message set_tempo tempo=560748 time=8>
<meta message set_tempo tempo=563380 time=7>
<meta message set_tempo tempo=566038 time=8>
<meta message set_tempo tempo=568720 time=7>
<meta message set_tempo tempo=571429 time=5>
<meta message set_tempo tempo=566038

info tracks: ['untitled1', '2']
tipo note off: False
cantidad eventos pedal: 302


,note_num,start,end,velocity,time_dif,track_name
0,39,391,422,85,391,2
1,27,392,414,80,1,2
2,46,526,646,79,134,2
3,54,629,723,82,103,2
4,63,703,736,76,74,2
...,...,...,...,...,...,...
1752,46,52246,52291,102,68,2
1753,54,52320,52544,96,74,2
1754,63,52418,52541,103,98,2
1755,70,52695,52712,108,277,2


In [7]:
## Calcula la duración de cada nota

tema_df.loc[:,'duration'] = tema_df.end - tema_df.start

In [8]:
## Parsea a enteros los valores numéricos

import numpy as np
for col in tema_df.loc[:,tema_df.columns!="track_name"].columns:
    tema_df[col] = pd.to_numeric(tema_df[col])

In [9]:
## agrega el nombre y octava de notas a la tabla

tema_df = pd.merge(tema_df, midi_notes,how='left',left_on='note_num',right_on='note_number').drop('note_number',axis=1)
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note
0,39,391,422,85,391,2,31,3,D#,D#3
1,27,392,414,80,1,2,22,2,D#,D#2
2,46,526,646,79,134,2,120,3,A#,A#3
3,54,629,723,82,103,2,94,4,F#,F#4
4,63,703,736,76,74,2,33,5,D#,D#5
...,...,...,...,...,...,...,...,...,...,...
1752,46,52246,52291,102,68,2,45,3,A#,A#3
1753,54,52320,52544,96,74,2,224,4,F#,F#4
1754,63,52418,52541,103,98,2,123,5,D#,D#5
1755,70,52695,52712,108,277,2,17,5,A#,A#5


In [10]:
tema_df.head(10)

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note
0,39,391,422,85,391,2,31,3,D#,D#3
1,27,392,414,80,1,2,22,2,D#,D#2
2,46,526,646,79,134,2,120,3,A#,A#3
3,54,629,723,82,103,2,94,4,F#,F#4
4,63,703,736,76,74,2,33,5,D#,D#5
5,70,773,857,84,70,2,84,5,A#,A#5
6,63,850,873,77,77,2,23,5,D#,D#5
7,54,910,989,84,60,2,79,4,F#,F#4
8,46,978,998,84,68,2,20,3,A#,A#3
9,39,1045,1073,66,67,2,28,3,D#,D#3


In [11]:
## Reemplaza outlyers de duración

notes_weight = pd.cut(tema_df.duration, 6)
print('duration notes before:')
display(notes_weight.value_counts())

outlyeras_duration = pd.DataFrame(tema_df.duration.quantile([0.05,0.95]))

mask_outlyers_lower = tema_df.duration < outlyeras_duration.duration[0.05]
print('lowers modificados:',mask_outlyers_lower.sum(), 'val:', outlyeras_duration.duration[0.05])
tema_df.loc[mask_outlyers_lower,'duration'] = outlyeras_duration.duration[0.05]

mask_outlyers_higher = tema_df.duration > outlyeras_duration.duration[0.95]
print('highers modificados:', mask_outlyers_higher.sum(), 'val:', outlyeras_duration.duration[0.95])
tema_df.loc[mask_outlyers_higher,'duration'] = outlyeras_duration.duration[0.95]

notes_weight = pd.cut(tema_df.duration, 6)
print('duration notes after:')
display(notes_weight.value_counts().head(7))

duration notes before:


(0.318, 114.667]      1587
(114.667, 228.333]     112
(228.333, 342.0]        41
(342.0, 455.667]        14
(569.333, 683.0]         2
(455.667, 569.333]       1
Name: duration, dtype: int64

lowers modificados: 80 val: 10.0
highers modificados: 88 val: 185.59999999999945
duration notes after:


(9.824, 39.267]       957
(39.267, 68.533]      424
(68.533, 97.8]        170
(156.333, 185.6]      120
(97.8, 127.067]        59
(127.067, 156.333]     27
Name: duration, dtype: int64

In [12]:
## Categoriza duración

cat_duration = pd.cut(tema_df.duration, 6, labels=['xxs','xs','s','m','l','xl'])
print('category duration notes:')
display(cat_duration.value_counts())
tema_df.loc[:,'cat_duration'] = cat_duration

category duration notes:


xxs    957
xs     424
s      170
xl     120
m       59
l       27
Name: duration, dtype: int64

In [13]:
## Categoriza dudación

cat_velocity = pd.cut(tema_df.velocity, 6, labels=['pp','p','m','mf','f','ff'])
print('category velocity notes:')
display(cat_velocity.value_counts())
tema_df.loc[:,'cat_velocity'] = cat_velocity

category velocity notes:


m     608
mf    604
p     323
f     161
pp     55
ff      6
Name: velocity, dtype: int64

In [14]:
## Describe, muchos de estos valores van a ser utiles como predictores

tema_describe = tema_df.describe()

In [15]:
## Cuantificación de notas:
## define cuales notas están tan cera que deberían estar en el mismo instante

unit_duplicate = tema_describe.loc['min','duration'] / 1.5
tema_df.loc[:,'note_simultaneous'] = tema_df.time_dif < unit_duplicate
print('ticks para quantizar: ',unit_duplicate)

ticks para quantizar:  6.666666666666667


In [16]:
## Reemplaza los tiempos de notas muy cercanas por notas simultaneas

tema_df2_mask = (tema_df.iloc[1:len(tema_df),:].note_simultaneous == True)
tema_df2_mask = tema_df2_mask
tema_df2_mask[0] = False

tema_df.loc[tema_df2_mask, 'start'] = np.NaN
tema_df.start = tema_df.start.fillna(method='ffill')
tema_df.start = tema_df.start.astype(int)
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous
0,39,391,422,85,391,2,31.0,3,D#,D#3,xxs,mf,False
1,27,391,414,80,1,2,22.0,2,D#,D#2,xxs,mf,True
2,46,526,646,79,134,2,120.0,3,A#,A#3,m,mf,False
3,54,629,723,82,103,2,94.0,4,F#,F#4,s,mf,False
4,63,703,736,76,74,2,33.0,5,D#,D#5,xxs,mf,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,46,52246,52291,102,68,2,45.0,3,A#,A#3,xs,f,False
1753,54,52320,52544,96,74,2,185.6,4,F#,F#4,xl,f,False
1754,63,52418,52541,103,98,2,123.0,5,D#,D#5,m,f,False
1755,70,52695,52712,108,277,2,17.0,5,A#,A#5,xxs,ff,False


In [17]:
## calcula la cantidad de ticks x segundo

ticks_por_seg = tema_df.end.max() / duracion_tema
print('Ticks por segundo:', ticks_por_seg)

Ticks por segundo: 213.55630116631724


In [18]:
## Convierte unidad de medida de timpo Ticks a segundos en cada nota

tema_df.loc[:,'segundo'] = tema_df.start / ticks_por_seg

In [19]:
## Calcula la cantidad de notas que existen en simultaneo

tema_df_simultaneous =  tema_df.start.value_counts()
tema_df_simultaneous_times = tema_df_simultaneous.loc[tema_df_simultaneous > 1].index.to_list()
print('cant. notas en simultaneo:',len(tema_df_simultaneous_times))

tema_df.note_simultaneous = False
tema_df.loc[:,'note_simultaneous'] = tema_df.start.apply(lambda x: True if x in tema_df_simultaneous_times else False) 

cant. notas en simultaneo: 500


In [20]:
## Implementacion DBSCAN para encontrar grupos de notas y segmentarlas

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X = tema_df[['start', 'note_num']]
X.loc[:,'start'] = X.start / 14
X.loc[:,'note_num'] = X.note_num / 0.2

X = X.values
print(X[0:5])

# Implementación de DBSCAN

dbscan = DBSCAN( eps=33, min_samples=2)
y_dbscan = dbscan.fit_predict(X)
y_dbscan

labels = dbscan.labels_
#unique_labels=set(labels);
tema_df.loc[:,'grupos'] = labels
print('cantidad de grupos:',pd.Series(labels).unique().shape[0])

[[ 27.92857143 195.        ]
 [ 27.92857143 135.        ]
 [ 37.57142857 230.        ]
 [ 44.92857143 270.        ]
 [ 50.21428571 315.        ]]
cantidad de grupos: 23


C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


import matplotlib.pyplot as plt
# Lista de los puntos core y máscara para graficar
core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True

fig, ax = plt.subplots(figsize=(80,80))

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for i,l in enumerate(unique_labels):
    if l ==-1:
        ax.scatter(X[labels==l,0], X[labels==l,1],c =colors[i], edgecolor='black', marker='o',s=40,label='Noise')
    else:
        ax.scatter(X[labels==l,0], X[labels==l,1],c =colors[i], edgecolor='blue', marker='o',s=40,label='Cluster '+str(l))
    
ax.legend()
ax.set_title('DBSCAN clustering');

In [21]:
## Concatena los grupos creados con DBSCAN con las cadenas de notas recibidas en cada track

tema_df.loc[:,'grupos_track'] = tema_df.grupos.astype(str) + "_" + tema_df.track_name
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous,segundo,grupos,grupos_track
0,39,391,422,85,391,2,31.0,3,D#,D#3,xxs,mf,True,1.830899,-1,-1_2
1,27,391,414,80,1,2,22.0,2,D#,D#2,xxs,mf,True,1.830899,-1,-1_2
2,46,526,646,79,134,2,120.0,3,A#,A#3,m,mf,False,2.463051,0,0_2
3,54,629,723,82,103,2,94.0,4,F#,F#4,s,mf,False,2.945359,0,0_2
4,63,703,736,76,74,2,33.0,5,D#,D#5,xxs,mf,False,3.291872,0,0_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,46,52246,52291,102,68,2,45.0,3,A#,A#3,xs,f,False,244.647429,17,17_2
1753,54,52320,52544,96,74,2,185.6,4,F#,F#4,xl,f,False,244.993942,17,17_2
1754,63,52418,52541,103,98,2,123.0,5,D#,D#5,m,f,False,245.452837,-1,-1_2
1755,70,52695,52712,108,277,2,17.0,5,A#,A#5,xxs,ff,False,246.749919,21,21_2


In [22]:
# Crea una mascara para limitar el ploteo en el tiempo

mask_tiempo_from = (tema_df.segundo >= plot_from_second)
mask_tiempo_to = (tema_df.segundo <= plot_to_second)

In [45]:
## Poltea las notas en el tiempo

import plotly.express as px

fig = px.scatter(data_frame = tema_df[mask_tiempo_from & mask_tiempo_to], x = "segundo", y = "note_num",
                 size = "duration", color = "grupos_track", opacity = 1, color_discrete_sequence=None)

fig.show()

In [24]:
## Imprime el dataset completo ordenado por grupos descartando el ruido
## Calcula el ruido

print('noise notes: %', round((tema_df.grupos == -1).sum() / tema_df.shape[0],2))
mask_noise = tema_df.grupos != -1
tema_df[mask_noise].sort_values(['grupos_track','start'],ascending=[True,True]).head(10)

noise notes: % 0.06


,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous,segundo,grupos,grupos_track
2,46,526,646,79,134,2,120.0,3,A#,A#3,m,mf,False,2.463051,0,0_2
3,54,629,723,82,103,2,94.0,4,F#,F#4,s,mf,False,2.945359,0,0_2
4,63,703,736,76,74,2,33.0,5,D#,D#5,xxs,mf,False,3.291872,0,0_2
6,63,850,873,77,77,2,23.0,5,D#,D#5,xxs,mf,False,3.980215,0,0_2
7,54,910,989,84,60,2,79.0,4,F#,F#4,s,mf,False,4.261171,0,0_2
8,46,978,998,84,68,2,20.0,3,A#,A#3,xxs,mf,False,4.579589,0,0_2
11,46,1129,1194,80,80,2,65.0,3,A#,A#3,xs,mf,False,5.286662,0,0_2
12,54,1191,1261,78,62,2,70.0,4,F#,F#4,s,mf,False,5.576984,0,0_2
13,63,1255,1275,71,64,2,20.0,5,D#,D#5,xxs,m,False,5.876670,0,0_2
15,63,1397,1419,61,71,2,22.0,5,D#,D#5,xxs,m,False,6.541600,0,0_2


In [25]:
## Shape final del dataset

notas_totales = tema_df.shape[0]
print('total notas:',notas_totales)

total notas: 1757


In [26]:
## indice de actividad (cantidad de notas) por tiempo
cant_eventos_individuales = (notas_totales - len(tema_df_simultaneous_times) / 2)
actividad_por_tiempo = cant_eventos_individuales / duracion_tema
print(round(cant_eventos_individuales,0),'/',round(duracion_tema,0))
print('densidad (notas por segundo):', round(actividad_por_tiempo,2))

1507.0 / 247.0
densidad (notas por segundo): 6.1


In [27]:
print('categoría velocity:')
display(tema_df.cat_velocity.value_counts())
print('categoría duration:')
display(tema_df.cat_duration.value_counts())
print('groups:') 
display(tema_df.grupos_track.value_counts())

categoría velocity:


m     608
mf    604
p     323
f     161
pp     55
ff      6
Name: cat_velocity, dtype: int64

categoría duration:


xxs    957
xs     424
s      170
xl     120
m       59
l       27
Name: cat_duration, dtype: int64

groups:


9_2     701
0_2     476
17_2    403
-1_2     98
13_2     16
16_2      6
11_2      6
5_2       5
12_2      5
20_2      5
7_2       4
6_2       4
1_2       4
19_2      3
4_2       3
18_2      3
10_2      3
2_2       2
8_2       2
3_2       2
21_2      2
14_2      2
15_2      2
Name: grupos_track, dtype: int64

In [28]:
tema_describe[['note_num','Octave','start','duration']].T

,count,mean,std,min,25%,50%,75%,max
note_num,1757.0,62.124644,13.087181,27.0,54.0,63.0,70.0,95.0
Octave,1757.0,4.667046,1.141030,2.0,4.0,5.0,5.0,7.0
start,1757.0,27740.779169,14026.194631,391.0,16142.0,27596.0,38338.0,52725.0
duration,1757.0,50.615139,45.030759,10.0,20.0,35.0,64.0,185.6


In [29]:
## Analiza proporciones de notas y duraciones mas repetidas

notes_weight = round(tema_df.note.value_counts(normalize=True) * 100,2)
print('notes most used:')
display(notes_weight.head(5))

notes_weight = round(tema_df.note_name.value_counts(normalize=True) * 100,2)
print('scale most used:')
display(notes_weight.head(7))
print('covarage with scale:', notes_weight.head(7).sum())

notes most used:


D#5    7.11
A#5    6.77
G#5    5.58
A#3    4.78
F#5    4.44
Name: note, dtype: float64

scale most used:


D#    17.64
A#    17.64
G#    13.26
F#    11.78
C#     8.65
B      8.14
F      7.97
Name: note_name, dtype: float64

covarage with scale: 85.08


In [30]:
avr_vertical_notes = tema_df.note_simultaneous.sum() / notas_totales
print('avr. notas simultaneas:', avr_vertical_notes)

avr. notas simultaneas: 0.7029026750142288


In [41]:
print('cantidad eventos por pedal:', cant_eventos_individuales / len(controls) if len(controls) > 5 else np.NaN)

cantidad eventos por pedal: 4.990066225165563
